## Declare Required Libraries

In [ ]:
import matplotlib.pyplot as plt
import folium
from nmea import reader as nmea

## Load NMEA file

In [ ]:
name = 'gps.txt'
nmeaDF = nmea.read_nmea(name)

## Check Message Types in file

In [ ]:
nmeaDF['COL1'].unique()

## Analyze NMEA messages

### Non analized message types

In [ ]:
nmeaDF.query("cksum.isnull()")['COL1'].unique()

### TXT messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+TXT$', regex=True)").loc[:, ['COL1','msg_num','msg_no','msg_type','msg_txt','cksum']]

### GSA Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+GSA', regex=True)").loc[:,['COL1','mode','status','sat_nos','pdop','hdop','vdop','sat_type','cksum']]

### GSV Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+GSV$', regex=True)").loc[:, ['COL1','sv_type','sv_msgs','sv_msg_no','sv_vlsibles','cksum']]

### GLL Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+GLL$', regex=True)").loc[:, ['COL1','latitude','longitude','time','status','mode','cksum']]

### VTG Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+VTG$', regex=True)").loc[:, ['COL1','true_azimuth','magnetic_azimuth','knot','kmh','mode','cksum']]

### ZDA Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+ZDA$', regex=True)").loc[:, ['COL1','time','date','tz_offset','cksum']]

### GGA Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+GGA$', regex=True) & mode!=0").loc[:, ['COL1','time','latitude','longitude','mode','satellites','hdop','altitude','geoid','age','cksum']]

#### GGA Messages: Charts

In [ ]:
ggaDF = nmeaDF.query('COL1 == "$GNGGA"')
ggaDF['valid'] = ggaDF['mode'].apply(lambda x: 0 if x=='0' else 1)
x = ggaDF.index.tolist()

titles = ['valid', 'latitude','longitude','mode','satellites','hdop','altitude','geoid']
fig, axes = plt.subplots(len(titles), 1)
fig.set_figwidth(16)
fig.set_figheight(6)
for i in range(len(titles)):
    axis = axes[i]
    axis.plot(x, ggaDF[titles[i]].tolist())
    axis.set_title(titles[i], loc='Left', pad=-14)
    axis.set_xlim(0, max(x))
    axis.tick_params(labelbottom=False, labelleft=True, labelright=False, labeltop=False)                    
axes[len(titles)-1].tick_params(labelbottom=True)
plt.show()

### RMC Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+RMC$', regex=True) & status=='A'").loc[:, ['COL1','datetime','status','latitude','longitude','knot','true_azimuth','magnetic_declination','mode','cksum']]

#### RMC Messages: Routes

In [ ]:
rmcDF = nmeaDF.query('COL1 == "$GNRMC" and COL3 =="A"')
locations = rmcDF[['latitude', 'longitude']].values.tolist()
locations
map = folium.Map(location=locations[0], tiles="OpenStreetMap", zoom_start=15)
folium.PolyLine(locations=locations).add_to(map)
map

#### RMC Messages: Charts

In [ ]:
rmcDF = nmeaDF.query('COL1 == "$GNRMC"')
rmcDF['valid'] = rmcDF['status'].apply(lambda x: 1 if x=='A' else 0)
x = rmcDF.index.tolist()

titles = ['valid', 'latitude','longitude','knot','true_azimuth']
fig, axes = plt.subplots(len(titles), 1)
fig.set_figwidth(16)
fig.set_figheight(6)
for i in range(len(titles)):
    axis = axes[i]
    axis.plot(x, rmcDF[titles[i]].tolist())
    axis.set_title(titles[i], loc='Left', pad=-14)
    axis.set_xlim(0, max(x))
    axis.tick_params(labelbottom=False, labelleft=True, labelright=False, labeltop=False)                    
axes[len(titles)-1].tick_params(labelbottom=True)
plt.show()